In [1]:
# For inline plotting
%matplotlib inline

# For auto reloading
%load_ext autoreload
%autoreload 2

In [3]:
import torch


if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.3.1  Device: cuda


# Profile the models for MNIST

In [17]:
from models import Mlp, BibdMlp, RandomSparseMlp
import numpy as np


input_dim = 28 * 28 * 1
output_dim = 10

model_list = []
model_list.append(Mlp(input_dim, output_dim).to(device))
model_list.append(BibdMlp(input_dim, output_dim).to(device))
model_list.append(RandomSparseMlp(input_dim, output_dim).to(device))
for model in model_list:
    print(model)

from thop import profile
import sys
sys.path.append('../bibd')
from bibd_layer import BibdLinear, RandomSparseLinear, generate_fake_bibd_mask


def count_model(model_to_count, x, y):
#     print(x.size())
    print(x)
    in_features = len(x)
    out_features = len(y)
    
    print('in_features = {}, out_features = {}'.format(in_features, out_features))
    
    # per output element
    total_mul = in_features
    total_add = in_features - 1
    num_elements = y.numel()
    total_ops = (total_mul + total_add) * num_elements
    # one zero weight, minus 2 ops
    total_ops -= (in_features * out_features - np.sum(generate_fake_bibd_mask(in_features, out_features))) * 2

    model_to_count.total_ops += torch.Tensor([int(total_ops)])

for model in model_list:
    input = torch.randn(1, 28, 28).to(device)
    flops, params = profile(model, inputs=(input, ), custom_ops={BibdLinear: count_model, RandomSparseLinear: count_model})
    print('Model: %s, Params(M): %.2f, FLOPs(M): %.2f' % (model.name, params / (1000 ** 2), flops / (1000 ** 2)))

Mlp(
  (fc1): Linear(in_features=784, out_features=112, bias=True)
  (fc2): Linear(in_features=112, out_features=56, bias=True)
  (fc3): Linear(in_features=56, out_features=10, bias=True)
)
BibdMlp(
  (bibd1): BibdLinear()
  (bibd2): BibdLinear()
  (fc3): Linear(in_features=56, out_features=10, bias=True)
)
RandomSparseMlp(
  (randomSparseLinear1): RandomSparseLinear()
  (randomSparseLinear2): RandomSparseLinear()
  (fc3): Linear(in_features=56, out_features=10, bias=True)
)
Register FLOP counter for module Linear(in_features=784, out_features=112, bias=True)
Register FLOP counter for module Linear(in_features=112, out_features=56, bias=True)
Register FLOP counter for module Linear(in_features=56, out_features=10, bias=True)
Model: MLP, Params(M): 0.09, FLOPs(M): 0.19
Register FLOP counter for module BibdLinear()
Register FLOP counter for module BibdLinear()
Register FLOP counter for module Linear(in_features=56, out_features=10, bias=True)
(tensor([[ 6.9076e-01,  8.6358e-01, -7.7553e-

In [18]:
import numpy as np
import sys
sys.path.append('../bibd')
from bibd_layer import generate_fake_bibd_mask


mask1 = generate_fake_bibd_mask(28*28, 28*4)
mask2 = generate_fake_bibd_mask(28*4, 56)
print(np.sum(mask1) + np.sum(mask2) + 56*10)

4266.0


In [19]:
mask1 = generate_fake_bibd_mask(32*32*3, 28*4)
mask2 = generate_fake_bibd_mask(28*4, 56)
print(np.sum(mask1) + np.sum(mask2) + 56*10)

7274.0
